In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd  # asi se suele importar Pandas
palette = 'colorblind'
sns.set_palette(palette); sns.set_color_codes(palette)  # Fija los nombres cortos para los colores según la paleta de seaborn

Esta notebook es una traducción y adaptación de [notebooks](https://github.com/fonnesbeck/Bios8366/tree/master/notebooks) creadas por Chris Fonnesbeck y de notebooks del libro [Python Data Science Handbook](https://github.com/jakevdp/PythonDataScienceHandbook) de Jake Vanderplas.

# Manipulación de datos y Pandas

**Pandas** es un paquete que Python que provee estructuras de datos rápidas, flexibles y expresivas diseñadas para trabajar con _rotulados_ y/o _relacionales_. Conceptualmente se pueden pensar como _arrays_ de NumPy donde las filas y colummnas están rótuladas. O de forma similar como una _planilla de cálculo_ bajo Python.

Asi como NumPy es una muy buena herramienta para trabajar con números, vectores, algebra lineal, etc. Pandas es adecuado para trabajar con:

* Datos tabulares y heterogeneos (flotantes, string, enteros, etc)
* Series temporales
* Los mismos datos que se pueden manipular con _arreglos_ de NumPy!

Key features:
    
- Easy handling of **missing data**
- **Size mutability**: columns can be inserted and deleted from DataFrame and higher dimensional objects
- Automatic and explicit **data alignment**: objects can be explicitly aligned to a set of labels, or the data can be aligned automatically
- Powerful, flexible **group by functionality** to perform split-apply-combine operations on data sets
- Intelligent label-based **slicing, fancy indexing, and subsetting** of large data sets
- Intuitive **merging and joining** data sets
- Flexible **reshaping and pivoting** of data sets
- **Hierarchical labeling** of axes
- Robust **IO tools** for loading data from flat files, Excel files, databases, and HDF5
- **Time series functionality**: date range generation and frequency conversion, moving window statistics, moving window linear regressions, date shifting and lagging, etc.

Por que es importante tener una herramienta como Pandas?

<img src="imagenes/analisis.jpeg"  width=300>

Pandas introduce fundamentalmente 3 nuevas estructuras de datos. La `Series` el `DataFrame`, y el `Index`. Empezemos por la primera de estas.

### Series

Una _Series_ de Pandas es un conjunto unidimensional de datos (similar a un _array_) acompañados de un índice que "rotula" a cada elemento del vector. Puede ser creada a partir de un array o tupla o lista.

In [2]:
conteo = pd.Series([632, 1638, 569, 115])
conteo

0     632
1    1638
2     569
3     115
dtype: int64

La primer columna es el índice y la segunda nuestros datos. Como omitimos indicar un índice Pandas asignó automáticamente una secuencia de enteros (empezando por 0). 

A partir de una serie es posible obtener solo el _array_ de NumPy "contenido" en ella.

In [3]:
conteo.values

array([ 632, 1638,  569,  115])

Como también es posible obtener solo el índice.

In [4]:
conteo.index

RangeIndex(start=0, stop=4, step=1)

Es importante notar que los arreglos de NumPy también tienen índices, solo que estos están implícitios y siemrpe son enteros comenzando desde el 0. En cambio los índices en Pandas son explícitos. Podemos asignar rotulos que tengan sentido según nuestros datos. Si nuestros datos representan la cantidad de diversas bacterias en una muestra, podríamos tener algo como:

In [5]:
bacteria = pd.Series([632, 1638, 569, 115], 
    index=['Firmicutes', 'Proteobacteria', 'Actinobacteria', 'Bacteroidetes'])

bacteria

Firmicutes         632
Proteobacteria    1638
Actinobacteria     569
Bacteroidetes      115
dtype: int64

Ahora que tenemos  rótulos por un lado y datos por el otro da la impresión que una serie se podría pensar también como una especie de diccionario! De hecho podemos usar los rótulos para referirnos directamente a valores en la serie.

In [6]:
bacteria['Actinobacteria']

569

O usando una sintaxis ligeramente más simple

In [7]:
bacteria.Actinobacteria

569

A diferencia de los diccionarios es posible hacer `slicing`

In [8]:
bacteria['Proteobacteria':]

Proteobacteria    1638
Actinobacteria     569
Bacteroidetes      115
dtype: int64

Al igual que con los arreglos de NumPy podemos usar booleanos para indexar una serie. Si quisieramos el conteo de  bacterias para todas aquellas cuyos nombres terminan en "bacteria" podríamos hacer:

In [9]:
bacteria[[name.endswith('bacteria') for name in bacteria.index]]

Proteobacteria    1638
Actinobacteria     569
dtype: int64

U operaciones como:

In [10]:
bacteria[bacteria>1000]

Proteobacteria    1638
dtype: int64

También es posible hacer uso del índice _implicito_ (como en listas y arreglos)

In [11]:
bacteria[1]

1638

We can give both the array of values and the index meaningful labels themselves:

In [12]:
bacteria.name = 'counts'
bacteria.index.name = 'phylum'
bacteria

phylum
Firmicutes         632
Proteobacteria    1638
Actinobacteria     569
Bacteroidetes      115
Name: counts, dtype: int64

Es posible operar con los valores de una serie manteniendo los rótulos sin modificar

In [13]:
np.log(bacteria)

phylum
Firmicutes        6.448889
Proteobacteria    7.401231
Actinobacteria    6.343880
Bacteroidetes     4.744932
Name: counts, dtype: float64

Como ya dijimos es posible pensar en una `Series` como si fuera una especie de diccionario, incluso podemos crear series a partir de diccionarios!

In [14]:
bacteria_dict = {'Firmicutes': 632, 'Proteobacteria': 1638, 'Actinobacteria': 569, 'Bacteroidetes': 115}
pd.Series(bacteria_dict)

Actinobacteria     569
Bacteroidetes      115
Firmicutes         632
Proteobacteria    1638
dtype: int64

Si observan con atención verán que Pandas ordenó alfabeticamente los indices en la `Series` (junto con sus valores asociados).

Si queremos añgun orden en particular podemos especificar los índices. Incluso podemos pasar rotulos para valores que no existen. En ese caso Pandas interpretará que tenemos datos faltantes (_missing data_) y lo indicará usando un tipo especial de _float_ `NaN` (not a number).

In [15]:
bacteria2 = pd.Series(bacteria_dict, index=['Cyanobacteria','Firmicutes','Proteobacteria','Actinobacteria'])
bacteria2

Cyanobacteria        NaN
Firmicutes         632.0
Proteobacteria    1638.0
Actinobacteria     569.0
dtype: float64

Es posible preguntarle a Pandas si tenemos valores faltantes

In [16]:
bacteria2.isnull()

Cyanobacteria      True
Firmicutes        False
Proteobacteria    False
Actinobacteria    False
dtype: bool

Los índices no son solo una conveniencia para manipular datos haciendo referencia a nombres que nos puede resultar más familiares o convenientes (comparado con recordar la posición de los datos). Los índices son usados para **alienar datos** al operar con más de una serie, por ej podríamos queres obtner el total de bacterias en dos conjuntos de datos.

In [17]:
bacteria

phylum
Firmicutes         632
Proteobacteria    1638
Actinobacteria     569
Bacteroidetes      115
Name: counts, dtype: int64

In [18]:
bacteria + bacteria2

Actinobacteria    1138.0
Bacteroidetes        NaN
Cyanobacteria        NaN
Firmicutes        1264.0
Proteobacteria    3276.0
dtype: float64

Como verán Pandas sumó solo los valores para  los cuales los índices de ambas `Series` coinciden, propagando además los valores faltantes (`NaN`). 

¿Cómo se compara esto con sumar dos arreglos de NumPy de distinta longitud?

## DataFrame

Al analizar datos es común que tengamos que trabajar con datos multivariados. Es decir con más de uan variable. Para esos casos sería útil tener algo como una `Series` donde para cada índice tengamos más de una columna de valores. Ese objeto se llama `DataFrame`.

Un `DataFrame` es una estructura de datos tabular que se puede pensar como una colección de `Series` (que comparten un mismo índice). También es posible pensar un `DataFrame` como una generalización de un arreglo de NumPy o de una diccionario. 

In [19]:
datos = pd.DataFrame({'conteo':[632, 1638, 569, 115, 433, 1130, 754, 555],
                     'paciente':[1, 1, 1, 1, 2, 2, 2, 2],
                     'phylum':['Firmicutes', 'Proteobacteria', 'Actinobacteria', 
    'Bacteroidetes', 'Firmicutes', 'Proteobacteria', 'Actinobacteria', 'Bacteroidetes']})
datos

,conteo,paciente,phylum
0,632,1,Firmicutes
1,1638,1,Proteobacteria
2,569,1,Actinobacteria
3,115,1,Bacteroidetes
4,433,2,Firmicutes
5,1130,2,Proteobacteria
6,754,2,Actinobacteria
7,555,2,Bacteroidetes


Lo primero que notamos es que `Jupyter` le pone un poco de estilo al `DataFrame`, y lo muestra como una tabla con algunas mejoras estéticas (comaprado con un montón de números como se ven arreglos, listas y series).

También podemos ver que contrario a un arreglo de NumPy en un `DataFrame` podemos tener datos de distinto tipo (enteros y _strings_ en este caso). Además se ve que las columnas están ordenadas alfabeticamente, podemos cambiar el orden indexando el `DataFrame` en el orden preferido.

In [20]:
datos[['paciente', 'phylum', 'conteo']]

,paciente,phylum,conteo
0,1,Firmicutes,632
1,1,Proteobacteria,1638
2,1,Actinobacteria,569
3,1,Bacteroidetes,115
4,2,Firmicutes,433
5,2,Proteobacteria,1130
6,2,Actinobacteria,754
7,2,Bacteroidetes,555


Los `DataFrame` tienen dos `index`, el mismo que ya vimos para las series que se corresponden con las filas y uno nuevo que se corresponde con las columnas

In [21]:
datos.columns

Index(['conteo', 'paciente', 'phylum'], dtype='object')

Es posible acceder a los valores de las columnas como con un diccionario

In [22]:
datos['conteo']

0     632
1    1638
2     569
3     115
4     433
5    1130
6     754
7     555
Name: conteo, dtype: int64

o por atributo (un atributo es el nombre que se le da a un dato o propiedad en programación orientada a objetos)

In [23]:
datos.conteo

0     632
1    1638
2     569
3     115
4     433
5    1130
6     754
7     555
Name: conteo, dtype: int64

Una posible fuente  de confusión es que la sintaxis que acabamos de ver devuelve filas en una `Series`, pero columnas en un `DataFrame`. Si queremos acceder a las filas de un `DataFrame` podemos hacerlo usando el atributo `ix`:

In [24]:
datos.ix[3]

conteo                115
paciente                1
phylum      Bacteroidetes
Name: 3, dtype: object

¿Que pasa si intentamos acceder a una fila usando la sintaxis `datos[3]`?

La serie que se obtieen al indexar un `DataFrame` es una _vista_ (_view_) del `DataFrame` y NO una copia. Por lo que hay que tener cuidado al manipularla, por ello Pandas nos devuelve una advertencia.

In [25]:
cont = datos.conteo
cont

0     632
1    1638
2     569
3     115
4     433
5    1130
6     754
7     555
Name: conteo, dtype: int64

In [26]:
cont[5] = 0
cont

/home/osvaldo/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0     632
1    1638
2     569
3     115
4     433
5       0
6     754
7     555
Name: conteo, dtype: int64

In [27]:
datos

,conteo,paciente,phylum
0,632,1,Firmicutes
1,1638,1,Proteobacteria
2,569,1,Actinobacteria
3,115,1,Bacteroidetes
4,433,2,Firmicutes
5,0,2,Proteobacteria
6,754,2,Actinobacteria
7,555,2,Bacteroidetes


Si queremos moficiar una `Series` que proviene de un `DataFrame` puede ser buena idea hacer una copia primero.

In [28]:
cont = datos.conteo.copy()
cont[5] = 1000
datos

,conteo,paciente,phylum
0,632,1,Firmicutes
1,1638,1,Proteobacteria
2,569,1,Actinobacteria
3,115,1,Bacteroidetes
4,433,2,Firmicutes
5,0,2,Proteobacteria
6,754,2,Actinobacteria
7,555,2,Bacteroidetes


Es posible crear columnas usando asignaciones

In [29]:
datos['year'] = 2013
datos

,conteo,paciente,phylum,year
0,632,1,Firmicutes,2013
1,1638,1,Proteobacteria,2013
2,569,1,Actinobacteria,2013
3,115,1,Bacteroidetes,2013
4,433,2,Firmicutes,2013
5,0,2,Proteobacteria,2013
6,754,2,Actinobacteria,2013
7,555,2,Bacteroidetes,2013


Pero para hacer esto no es posible usar la sintaxiis de  atributo

In [30]:
datos.tratamiento = 1

Aunque esto crea el atributo y le asgina el valor 1

In [31]:
datos.tratamiento

1

Este cambio no se ve reflejado en el `DataFrame`

In [32]:
datos

,conteo,paciente,phylum,year
0,632,1,Firmicutes,2013
1,1638,1,Proteobacteria,2013
2,569,1,Actinobacteria,2013
3,115,1,Bacteroidetes,2013
4,433,2,Firmicutes,2013
5,0,2,Proteobacteria,2013
6,754,2,Actinobacteria,2013
7,555,2,Bacteroidetes,2013


Podemos agregar una `Series` como una nueva columna en un `DataFrame`, el resultado dependerá de los índices de ambos objetos.

In [33]:
tratamiento = pd.Series([0]*4 + [1]*2)
tratamiento

0    0
1    0
2    0
3    0
4    1
5    1
dtype: int64

In [34]:
datos['tratamiento'] = tratamiento
datos

,conteo,paciente,phylum,year,tratamiento
0,632,1,Firmicutes,2013,0.0
1,1638,1,Proteobacteria,2013,0.0
2,569,1,Actinobacteria,2013,0.0
3,115,1,Bacteroidetes,2013,0.0
4,433,2,Firmicutes,2013,1.0
5,0,2,Proteobacteria,2013,1.0
6,754,2,Actinobacteria,2013,NaN
7,555,2,Bacteroidetes,2013,NaN


¿Que pasa si intentamos agregar una nueva columna que no sea una serie y cuyo longitud no coincida con la del `DataFrame`?

In [35]:
datos['mes'] = ['enero'] * len(datos)
datos

,conteo,paciente,phylum,year,tratamiento,mes
0,632,1,Firmicutes,2013,0.0,enero
1,1638,1,Proteobacteria,2013,0.0,enero
2,569,1,Actinobacteria,2013,0.0,enero
3,115,1,Bacteroidetes,2013,0.0,enero
4,433,2,Firmicutes,2013,1.0,enero
5,0,2,Proteobacteria,2013,1.0,enero
6,754,2,Actinobacteria,2013,NaN,enero
7,555,2,Bacteroidetes,2013,NaN,enero


Podemos usar `del` para eliminar columnas de la misma forma que en diccionarios.

In [36]:
del datos['mes']
datos

,conteo,paciente,phylum,year,tratamiento
0,632,1,Firmicutes,2013,0.0
1,1638,1,Proteobacteria,2013,0.0
2,569,1,Actinobacteria,2013,0.0
3,115,1,Bacteroidetes,2013,0.0
4,433,2,Firmicutes,2013,1.0
5,0,2,Proteobacteria,2013,1.0
6,754,2,Actinobacteria,2013,NaN
7,555,2,Bacteroidetes,2013,NaN


Es posible _extaer_ los datos de un `DataFrame` en forma de arreglo de NumPy.

In [37]:
datos.values

array([[632, 1, 'Firmicutes', 2013, 0.0],
       [1638, 1, 'Proteobacteria', 2013, 0.0],
       [569, 1, 'Actinobacteria', 2013, 0.0],
       [115, 1, 'Bacteroidetes', 2013, 0.0],
       [433, 2, 'Firmicutes', 2013, 1.0],
       [0, 2, 'Proteobacteria', 2013, 1.0],
       [754, 2, 'Actinobacteria', 2013, nan],
       [555, 2, 'Bacteroidetes', 2013, nan]], dtype=object)

Fijense que el `dtype` del arreglo es `object`. Esto se debe a la mezcla de enteros _strings_ y flotantes (`Nan`). El `dtype` es elegido por Pandas automaticamente de forma tal de acomodar todos los tipos de valores presentes en el `DataFrame`.

## Index

La última estructura de datos que nos queda ver es `Index`, la cual en realidad la venimos usando desde el principio de este capítulo. Solo que ahora hablaremos de ella de forma un poco más explícita.

In [38]:
datos.index

RangeIndex(start=0, stop=8, step=1)

Los `Index` son inmutables

In [39]:
#datos.index[0] = 15

Esto está pensado para permitir que los `Index` se compartan entre objetos sin riesgo de que se modifiquen en algún momento.

In [40]:
bacteria2.index = bacteria.index

In [41]:
bacteria2

phylum
Firmicutes           NaN
Proteobacteria     632.0
Actinobacteria    1638.0
Bacteroidetes      569.0
dtype: float64

## Importing data

En pirncipio es posible usar Python para leer cualquier archivo que uno desee, pero para el trabajo rutinario de un científico de datos esto es una opción de _demasiado bajo nivel_, NumPy provee de algunas funciones (como `genfromtxt` y `loadtxt`) para leer archivos que funcionan bastante bien para archivos relativamente simples. Pandas ofrece funciones  más vestátiles  y robustas para cuando nos encontramos con archivos _no tan simples_.

Empecemos leyendo un archivo en formato csv (comma separated values)

In [42]:
!head datos/microbiome.csv  # este es un comando de linux que nos permite ver las primeras lineas de un archivo

Taxon,Paciente,Grupo,Tejido,Heces
Firmicutes,1,0,136,4182
Firmicutes,2,1,1174,703
Firmicutes,3,0,408,3946
Firmicutes,4,1,831,8605
Firmicutes,5,0,693,50
Firmicutes,6,1,718,717
Firmicutes,7,0,173,33
Firmicutes,8,1,228,80
Firmicutes,9,0,162,3196


Pandas ofrece una función llamada `read_csv` ideal para leer este tipo de datos:

In [43]:
mb = pd.read_csv('datos/microbiome.csv')
mb.head()  # le pedimos a Pandas que nos nuestre solo las primeras lineas, esto es similar al "head" de Linux!

,Taxon,Paciente,Grupo,Tejido,Heces
0,Firmicutes,1,0,136,4182
1,Firmicutes,2,1,1174,703
2,Firmicutes,3,0,408,3946
3,Firmicutes,4,1,831,8605
4,Firmicutes,5,0,693,50


Por defecto `read_csv` usará la primer linea del archivo como encabezado (_header_). Este comportamiento lo podemos modificar usando el argumento `header`.

In [44]:
pd.read_csv('datos/microbiome.csv', header=None).head()

,0,1,2,3,4
0,Taxon,Paciente,Grupo,Tejido,Heces
1,Firmicutes,1,0,136,4182
2,Firmicutes,2,1,1174,703
3,Firmicutes,3,0,408,3946
4,Firmicutes,4,1,831,8605


Por defecto `read_csv` usa `,` como separadores, pero es posible modificar este comportamiento usando el argumento `sep`. Un caso muy común es el de archivos que tienen una cantidad variable de espacios en blanco. En esos casos podemos usar la [expresión regular](https://docs.python.org/2/library/re.html):
 
    sep='\s+'

Que quiere decir use como separador 1 o más espacios en blanco. Otro caso común son archivos separados por tabulaciones, en ese caso podemos usar `\t`.

Ahora probemos con usar las dos primeras columnas como índices.

In [45]:
mb = pd.read_csv('datos/microbiome.csv', index_col=['Paciente','Taxon'])
mb.head()

,,Grupo,Tejido,Heces
Paciente,Taxon,,,
1,Firmicutes,0,136,4182
2,Firmicutes,1,1174,703
3,Firmicutes,0,408,3946
4,Firmicutes,1,831,8605
5,Firmicutes,0,693,50


Si queremos omitir datos (por ejemplo datos mal tomados), podemos indicaselo a Pandas usando el argumento `skiprows`:

In [46]:
pd.read_csv('datos/microbiome.csv', skiprows=[3,4,6]).head()

,Taxon,Paciente,Grupo,Tejido,Heces
0,Firmicutes,1,0,136,4182
1,Firmicutes,2,1,1174,703
2,Firmicutes,5,0,693,50
3,Firmicutes,7,0,173,33
4,Firmicutes,8,1,228,80


También podemos indicarque que solo queremos importar unas pocas columnas, lo que puede ser muy útil cuando estamos haciendo pruebas y explorando los datos y queremos evitar importar un larga lista de datos.

In [47]:
pd.read_csv('datos/microbiome.csv', nrows=4)

,Taxon,Paciente,Grupo,Tejido,Heces
0,Firmicutes,1,0,136,4182
1,Firmicutes,2,1,1174,703
2,Firmicutes,3,0,408,3946
3,Firmicutes,4,1,831,8605


Es común al analizar datos encontrarnos con datos faltantes. Las razones son variadas desde errores de transcripción, errores en la toma de muestra, observaciones incompletas, etc. En algunos casos estos datos faltantes quedan registrados simplemente como _huecos_ en los datos o usando alguno indicadores de datos faltantes como `NA` o `NULL` o _valores sentinelas_ como `999`, `-9999` (es decir valores que están _claramente_ fuera del rango de los datos). 

In [48]:
!head datos/microbiome_missing.csv

Taxon,Patient,Tissue,Stool
Firmicutes,1,632,305
Firmicutes,2,136,4182
Firmicutes,3,,703
Firmicutes,4,408,3946
Firmicutes,5,831,8605
Firmicutes,6,693,50
Firmicutes,7,718,717
Firmicutes,8,173,33
Firmicutes,9,228,NA


In [49]:
mbm = pd.read_csv('datos/microbiome_missing.csv')
mbm.head(14)

,Taxon,Patient,Tissue,Stool
0,Firmicutes,1,632,305.0
1,Firmicutes,2,136,4182.0
2,Firmicutes,3,NaN,703.0
3,Firmicutes,4,408,3946.0
4,Firmicutes,5,831,8605.0
5,Firmicutes,6,693,50.0
6,Firmicutes,7,718,717.0
7,Firmicutes,8,173,33.0
8,Firmicutes,9,228,NaN
9,Firmicutes,10,162,3196.0


En el ejemplo anterior Pandas reconoció correctamente a `NA` y a un campo vacío como datos faltantes, pero pasó por alto a `?` y a `-99999`. Es facil pasar por alto estos errores, por lo que siemrpe es buena idea hacer gráficos de los datos y resúmenes como el siguiente:

In [50]:
mbm.describe()

,Patient,Stool
count,75.000000,74.000000
mean,8.000000,-619.283784
std,4.349588,11801.273013
min,1.000000,-99999.000000
25%,4.000000,12.500000
50%,8.000000,79.500000
75%,12.000000,658.500000
max,15.000000,8605.000000


Se puede ver que el conteo para `Paciente` y `Heces` no coinciden, que el valor más pequeño para `Heces` es un número negativo cuando debería ser mator o igual a cero. Y vemos que no tenémos descripción para `Tejido`! ¿Se te ocurre por que falta la columna para `Tejido`?

Para especificar valores addicionales a considerar como datos faltantes usamos `na_values`.

In [51]:
mbm = pd.read_csv('datos/microbiome_missing.csv', na_values=['?', -99999])

In [52]:
mbm.describe()

,Patient,Tissue,Stool
count,75.000000,73.000000,73.000000
mean,8.000000,984.315068,742.082192
std,4.349588,1840.338155,1467.675342
min,1.000000,0.000000,0.000000
25%,4.000000,109.000000,14.000000
50%,8.000000,310.000000,83.000000
75%,12.000000,831.000000,661.000000
max,15.000000,12044.000000,8605.000000


Si fuese necesario especificar valores distintos para distintas columnas es posible pasar un diccionario a `na_values`, indicando los nombres de las columnas y los valores a usar como indicadores. Este es un buen momento para que pruebes como hacer esto antes de seguir con la nueva sección.

### Otros formatos

Pandas ofrece la capacidad de leer varios otros formatos incluyendo archivos `xls`, `xlsx`, `JSON`, `XML`, `HDF5`, etc. PAra más información leer la documentación de Pandas o [Python for Data Analysis](http://shop.oreilly.com/product/0636920023784.do).

## Indexado y selección en Series

El idexado y selección en Pandas es muy parecido al de NumPy.

Como ya vimos las series se pueden pensar como diccionarios o como arreglos unidimensionales. Los siguientes ejemplos muestra que ambas ideas son útiles.

In [53]:
datos = pd.Series([0, .25, .5, .75], index=['a', 'b', 'c', 'd'])

In [54]:
datos['b']

0.25

In [55]:
'e' in datos

False

In [56]:
datos.keys()

Index(['a', 'b', 'c', 'd'], dtype='object')

In [57]:
[_ for _ in datos.items()]

[('a', 0.0), ('b', 0.25), ('c', 0.5), ('d', 0.75)]

In [58]:
datos['e'] = 1.
datos

a    0.00
b    0.25
c    0.50
d    0.75
e    1.00
dtype: float64

Al indexar con un índice implícito el último índice NO se incluye. Esto es lo que esperamos de listas, tuplas, arreglos etc.

In [59]:
datos[1:3]  

b    0.25
c    0.50
dtype: float64

Por lo que puede resultar confuso que al indexar con un índice explícito el último índice se incluye!

In [60]:
datos['b':'d']

b    0.25
c    0.50
d    0.75
dtype: float64

In [61]:
datos[datos > .5]

d    0.75
e    1.00
dtype: float64

In [62]:
datos[['e', 'a']]

e    1.0
a    0.0
dtype: float64

### Indexadores: loc e iloc

La presencia de índices implícitos y explícitos puede ser una fuente de gran confusión al usar Pandas. Veamos, que sucede cuando tenemosuna serie con índices explícitos que son enteros.

In [63]:
datos = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])

Pandas usará el índice explítico al indexar

In [64]:
datos[1]

'a'

Pero el implícito al tomar rebanadas!

In [65]:
datos[1:3]

3    b
5    c
dtype: object

A fin de evitar confusiones Pandas provee de algunos métodos especiales para indexar. El primero de ellos es `loc` que permite hacer las operaciones de indexado/rebanado usando SIEMPRE el índice explícito.

In [66]:
datos.loc[1]

'a'

In [67]:
datos.loc[1:3]

1    a
3    b
dtype: object

la contraparte de `loc` es `iloc` quien siemrpe usa el índice  **i**mplícito.

In [68]:
datos.iloc[1]

'b'

In [69]:
datos.iloc[1:3]

3    b
5    c
dtype: object

Siguiendo el zen de Python que nos dice que “explícito es mejor que implícito". La recomendación general es usar `loc` e `iloc`. Ya que hace explícita la intención del código lo que ayuda a una más facil lectura y a reducir la posibilidad de errores.

In [70]:
datos.ix[1]

'a'

In [71]:
datos.ix[1:3]

1    a
3    b
dtype: object

## Indexado y selección en DataFrames

In [72]:
pob = {'BSAS':15594428,'Córdoba':3304825, 'Santa Fe':3300736, 'CABA':2891082, 'Mendoza':1741610}
sup = {'BSAS':307571,'Córdoba':165321, 'Santa Fe':133007, 'CABA':202, 'Mendoza':148827}

datos = pd.DataFrame({'sup':sup, 'pob':pob})
datos

,pob,sup
BSAS,15594428,307571
CABA,2891082,202
Córdoba,3304825,165321
Mendoza,1741610,148827
Santa Fe,3300736,133007


Como ya vimos es posible acceder a cada serie que forma un DataFrame usando una sintáxis simialr a la de los diccionarios

In [73]:
datos['sup']

BSAS        307571
CABA           202
Córdoba     165321
Mendoza     148827
Santa Fe    133007
Name: sup, dtype: int64

O usando una sintáxis de atributo

In [74]:
datos.sup

BSAS        307571
CABA           202
Córdoba     165321
Mendoza     148827
Santa Fe    133007
Name: sup, dtype: int64

La sintátix de atributo es solo _azucar sintáctico_ y podemos comprobar que devuelve exactamente el mismo objeto.

In [75]:
datos.sup is datos['sup']

True

Esta sintáxis no funciona para todos los casos, Algunos casos donde fallará es si la columna continene espacios o si el nombre de la columna entra en conflicto con algún método existente para DataFrames, por ejemplo no sería raro que llamaramos a una columna con alguno de estos nombres `all`, `cov`, `index`, `mean`.

Como ya vimos agregar una nueva columna a un DataFrame es similar a agregar un nuevo elemento a un diccionario.

In [76]:
datos['dens'] = datos['pob'] / datos['sup']
datos

,pob,sup,dens
BSAS,15594428,307571,50.701880
CABA,2891082,202,14312.287129
Córdoba,3304825,165321,19.990352
Mendoza,1741610,148827,11.702245
Santa Fe,3300736,133007,24.816258


Como ya vimos en la introducción a DataFrames, es posible ver estas estructuras de datos como arreglos de NumPy. Esto se hace evidente al quedarnos solo con los _valores_ de un DataFrame.

In [77]:
datos.values

array([[  1.55944280e+07,   3.07571000e+05,   5.07018802e+01],
       [  2.89108200e+06,   2.02000000e+02,   1.43122871e+04],
       [  3.30482500e+06,   1.65321000e+05,   1.99903521e+01],
       [  1.74161000e+06,   1.48827000e+05,   1.17022449e+01],
       [  3.30073600e+06,   1.33007000e+05,   2.48162578e+01]])

Pero las similitudes son más amplias, por ejemplo podemos transponer un DataFrame al igual que un _arreglo_.

In [78]:
datos.T

,BSAS,CABA,Córdoba,Mendoza,Santa Fe
pob,1.559443e+07,2.891082e+06,3.304825e+06,1.741610e+06,3.300736e+06
sup,3.075710e+05,2.020000e+02,1.653210e+05,1.488270e+05,1.330070e+05
dens,5.070188e+01,1.431229e+04,1.999035e+01,1.170224e+01,2.481626e+01


Las similitudes de un DataFrame entre un diccionario y un arreglo entran en conflicto al querer indexarlo. Veamos, no sorprende que al pasar un solo índice

In [79]:
datos.values[0]

array([  1.55944280e+07,   3.07571000e+05,   5.07018802e+01])

obtengamos la primer fila, esto es precisamente lo que se espera de un arreglo de NumPy. Pero hay que notar que al pasar un solo _índice_ a un DataFrame, lo que se obtiene es una columna y no una fila!

In [80]:
datos['sup']

BSAS        307571
CABA           202
Córdoba     165321
Mendoza     148827
Santa Fe    133007
Name: sup, dtype: int64

Por ello también existen indexadores especiales para DataFrames, `loc` e `iloc` ya los  conocemos de la sección anterior. Recordemos `loc` usa el índice explícito. 

In [81]:
datos.loc[:'Córdoba']

,pob,sup,dens
BSAS,15594428,307571,50.701880
CABA,2891082,202,14312.287129
Córdoba,3304825,165321,19.990352


Y ya que tenemos dos dimensiones para indexar podemos hacer selecciones como la siguiente:

In [82]:
datos.loc[:'Córdoba', 'sup':]

,sup,dens
BSAS,307571,50.701880
CABA,202,14312.287129
Córdoba,165321,19.990352


Al usar `iloc` especificamos el índice implicito. Por lo que para obtener el mismo resultado que en la celda anterior hacemos:

In [83]:
datos.iloc[:3, 1:]

,sup,dens
BSAS,307571,50.701880
CABA,202,14312.287129
Córdoba,165321,19.990352


¿Y si quisieramos mezclar índices explítios e implícitos? En ese caso podemos usar `ix`.

In [84]:
datos.ix[:3, :'sup']

,pob,sup
BSAS,15594428,307571
CABA,2891082,202
Córdoba,3304825,165321


El problema con `ix` es que si usamos enteros como índices explícitos, podemos terminar con el mismo tipo de confusiones que discutimos en la sección anterior. 

Como empezamos a ver en el capítulo sobre NumPy, es posible seleccionar subconjuntos de datos de forma muy versatil, estas capacidades se extienden a los DataFrames.

In [85]:
datos.loc[datos.dens > 50, ['sup', 'dens']]

,sup,dens
BSAS,307571,50.701880
CABA,202,14312.287129


## Funciones Universales

Una de las características más valiosas de NumPY es la posiblidad de vectorizar código, evitando escribir _loops_, al realizar operaciones como sumas, multiplicaciones, logaritmos, etc. Pandas hereda de NumPy esta capacidad y la adapta de dos formas:

1. Para operaciones unarias al aplicar funciones univerales se preserva el índice, es decir solo se aplican las operaciones a los valores y no a los _rótulos_.
2. Para operaciones binarias, las mismas se realizan sobre los índices alineados

Esto facilita el realizar operaciones que implican combinar datos de distintas fuentes, algo que puede no ser tan simple al usar NumPy.

Veamos un ejemplo al operar con dos series a fin de calcular la densidad de población

In [86]:
pob = pd.Series({'BSAS':15594428,'Córdoba':3304825, 'Santa Fe':3300736, 'CABA':2891082, 'San Luis':476351}, name='pob')
sup = pd.Series({'BSAS':307571,'Córdoba':165321, 'Santa Fe':133007, 'CABA':202, 'Mendoza':148827}, name='sub')

pob / sup

BSAS           50.701880
CABA        14312.287129
Córdoba        19.990352
Mendoza              NaN
San Luis             NaN
Santa Fe       24.816258
dtype: float64

El resultado es una `Series` donde el índice corresponde a la _unión_ de los índices de `sup` y `pop`. Como para `san Luis` tenemos la población pero no la superficie y para `Mendoza` tenemos la  superficie pero no la población. El resultado es que en la nueva `Series` obtenemos `NaN` para estas dos provincias. 

Es posible cambiar este comportamiento al usar el método `.div()` el cual equivale a la división de la celda anterior, pero ahora podemos indicar que cambie `NaN` por otro valor.

In [87]:
pob.div(sup, fill_value=0)

BSAS        5.070188e+01
CABA        1.431229e+04
Córdoba     1.999035e+01
Mendoza     0.000000e+00
San Luis             inf
Santa Fe    2.481626e+01
dtype: float64

El cambio se realiza antes de realizar la operación, por eso obtenemos `0` para `Mendoza` e `inf` para San Luis. Una variante sería hacer la operación y luego cambiar los `NaN` por cualquier otro valor (incluso uno con muy poco sentido).

In [88]:
(pob / sup).fillna('ϵ')

BSAS        50.7019
CABA        14312.3
Córdoba     19.9904
Mendoza           ϵ
San Luis          ϵ
Santa Fe    24.8163
dtype: object

En los DataFrames el _alineamiento_ tiene en cuenta  tanto las columnas comos los índices.

In [89]:
A = pd.DataFrame(np.arange(1, 5).reshape(2, 2), columns=['A', 'B'])
A

,A,B
0,1,2
1,3,4


In [90]:
B = pd.DataFrame(np.arange(10, 19).reshape(3, 3), columns=['B', 'A', 'C'])
B

,B,A,C
0,10,11,12
1,13,14,15
2,16,17,18


In [91]:
A + B

,A,B,C
0,12.0,12.0,NaN
1,17.0,17.0,NaN
2,NaN,NaN,NaN


Como habrás podido ver los índices quedan alineados correctamente independientemente de su orden en los objetos originales.

Como en el caso de `Series`, podemos utilizar las el método aritmético del objeto y pasar cualquier valor de relleno que deseemos, por ejemplo la media de todos los valores en A.

In [92]:
fill = A.values.mean()
A.add(B, fill_value=fill)

,A,B,C
0,12.0,12.0,14.5
1,17.0,17.0,17.5
2,19.5,18.5,20.5


Por supuesto también podemos combinar operacioens entre `Series` y `DataFrames` y Pandas se ocupará de mantener los "rótulos" alineados.

In [93]:
B - B.iloc[0]

,B,A,C
0,0,0,0
1,3,3,3
2,6,6,6


La operación anterior se realizó a lo largo de las filas (como se esperaría en NumPy), si quisieramos hacerlo a lo largo de las columnas:

In [94]:
B.subtract(B['B'], axis=0)

,B,A,C
0,0,1,2
1,0,1,2
2,0,1,2


## Datos Faltantes